In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import torch

from torch import nn
from torch import optim
from torch.utils.data import DataLoader

In [2]:
from ignite.engine import Events, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [3]:
from cbow.dataset import CBOWDataset, DataContainer
from cbow.model import CBOWClassifier, ModelContainer
from cbow.consts import consts
from cbow.trainer import IgniteTrainer
consts

Namespace(batch_size=128, checkpointer_name='cbow_classifier', checkpointer_prefix='cbow', device='cuda:3', early_stopping_criteria=5, embedding_size=50, learning_rate=0.0001, metric_file=PosixPath('../data/books/work_dir/metrics.csv'), model_dir=PosixPath('../data/books/work_dir/models'), num_epochs=100, path=PosixPath('../data/books'), proc_dataset_csv=PosixPath('../data/books/frankenstein_with_splits.csv'), save_every=2, save_total=5, vectorizer_file=PosixPath('../data/books/work_dir/vectorizer.json'), work_dir=PosixPath('../data/books/work_dir'))

In [4]:
df = pd.read_csv(consts.proc_dataset_csv)
dc = DataContainer(df, consts.vectorizer_file, consts.batch_size, is_load=True)

In [5]:
classifier = CBOWClassifier(dc.vocabulary_size, consts.embedding_size)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=consts.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1)

mc = ModelContainer(classifier, optimizer, loss_func, scheduler)

In [6]:
pbar = ProgressBar(persist=True)
metrics = {'accuracy': Accuracy(), 'loss': Loss(loss_func)}

In [7]:
consts.num_epochs=2

In [8]:
ig = IgniteTrainer(mc, dc, consts, pbar, metrics)

In [9]:
ig.run()

Epoch [1/2]: [496/496] 100%|██████████, loss=9.70e+00 [00:07<00:00]
Epoch [2/2]: [11/496]   2%|▏         , loss=9.66e+00 [00:00<00:06]

Epoch: 1
Training - Loss: 8.792, Accuracy: 0.037
Validation - Loss: 8.837, Accuracy: 0.037


Epoch [2/2]: [496/496] 100%|██████████, loss=8.68e+00 [00:07<00:00]


Epoch: 2
Training - Loss: 7.915, Accuracy: 0.053
Validation - Loss: 8.039, Accuracy: 0.054
